In [56]:
import pandas as pd
import numpy as np
import datetime
import json
import glob
from fuzzywuzzy import fuzz
from haversine import haversine

In [4]:
new_store_list=pd.read_csv("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20190311/new_store_appared.csv",dtype=str)

new_store_list['address_str']=new_store_list['Address']+", "+new_store_list['city']+", "+new_store_list['state']+", "+new_store_list['zip']
new_store_list=new_store_list[['storenumber','address_str']].drop_duplicates()



In [12]:
latest_SG_brand_info=pd.read_csv("/home/jian/Docs/SafeGraph/places_20190228/brand_info/part-00000-tid-5936742275909629561-4034b3da-5c73-494f-b611-27b92f5ac7cc-571-c000.csv.gz",
                                dtype=str,compression="gzip",sep="|")
[x for x in latest_SG_brand_info['brand_name'].unique().tolist() if "moothie" in x]

['Emerald City Smoothie',
 'Mr. Smoothie',
 'Planet Smoothie',
 'Pulp Juice and Smoothie Bar',
 'Smoothie King',
 'Tropical Smoothie Café']

In [17]:
POI_data=glob.glob("/home/jian/Docs/SafeGraph/places_20190228/poi/*csv.gz")

In [21]:
all_SG_SK_POI=pd.DataFrame()

for file in POI_data:
    df=pd.read_csv(file,dtype=str,compression="gzip",sep="|")
    df=df[df['brands']=="Smoothie King"]
    all_SG_SK_POI=all_SG_SK_POI.append(df)
    print(datetime.datetime.now(),file)

2019-03-11 16:37:17.075931 /home/jian/Docs/SafeGraph/places_20190228/poi/part-00006-tid-8303730837246125361-bea94277-caea-404c-90f6-6a1d6288e138-569-c000.csv.gz
2019-03-11 16:37:29.030787 /home/jian/Docs/SafeGraph/places_20190228/poi/part-00000-tid-8303730837246125361-bea94277-caea-404c-90f6-6a1d6288e138-563-c000.csv.gz
2019-03-11 16:37:40.594701 /home/jian/Docs/SafeGraph/places_20190228/poi/part-00004-tid-8303730837246125361-bea94277-caea-404c-90f6-6a1d6288e138-567-c000.csv.gz
2019-03-11 16:37:52.182384 /home/jian/Docs/SafeGraph/places_20190228/poi/part-00003-tid-8303730837246125361-bea94277-caea-404c-90f6-6a1d6288e138-566-c000.csv.gz
2019-03-11 16:38:03.451803 /home/jian/Docs/SafeGraph/places_20190228/poi/part-00007-tid-8303730837246125361-bea94277-caea-404c-90f6-6a1d6288e138-570-c000.csv.gz
2019-03-11 16:38:15.062153 /home/jian/Docs/SafeGraph/places_20190228/poi/part-00002-tid-8303730837246125361-bea94277-caea-404c-90f6-6a1d6288e138-565-c000.csv.gz
2019-03-11 16:38:26.624044 /home/j

In [25]:
all_SG_SK_POI=all_SG_SK_POI.reset_index()
del all_SG_SK_POI['index']
print(all_SG_SK_POI.shape)
all_SG_SK_POI.head(2)

(852, 18)


,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,state,zip_code,open_hours,polygon_wkt,polygon_class,phone_number
0,sg:5efcd8654a3e46ad831c773ea76529df,NaN,SG_BRAND_19800a6b4d6fb62e6752413f3ac722ef,Smoothie King,Smoothie King,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,722515,35.148282,-80.833469,4736 sharon road,charlotte,nc,28210,"{ ""Mon"": [[""7:00"", ""21:00""]], ""Tue"": [[""7:00"",...","POLYGON ((-80.833462 35.148348, -80.833385 35....",SHARED_POLYGON,17049005550
1,sg:79312f30ff1543708c0cfbbd028dc5a3,NaN,SG_BRAND_19800a6b4d6fb62e6752413f3ac722ef,Smoothie King,Smoothie King,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,722515,25.685367,-80.398544,12520 south west 88th street,miami,fl,33186,"{ ""Mon"": [[""7:00"", ""22:00""]], ""Tue"": [[""7:00"",...","POLYGON ((-80.398614 25.685253, -80.39863 25.6...",SHARED_POLYGON,13055987004


In [31]:
all_SG_SK_POI['SG_address_str']=all_SG_SK_POI['street_address']+", "+all_SG_SK_POI['city']+", "+\
                                all_SG_SK_POI['state']+", "+all_SG_SK_POI['zip_code']

In [36]:
def match_best_str(str_value,list_2):
    str_value=str_value.lower()
    ratio_max=0
    best_match=""
    for SG_add in list_2:
        ratio=fuzz.partial_ratio(str_value,SG_add)
        if ratio>ratio_max:
            ratio_max=ratio
            best_match=SG_add
    return [best_match,ratio_max]

In [45]:
all_SG_SK_POI['matched_new_stores']=np.nan
all_SG_SK_POI['highest_score']=0
for i in range(len(all_SG_SK_POI)):
    max_ratio=0
    best_match=np.nan
    store_SG=all_SG_SK_POI['SG_address_str'][i]
    for new_store_add in new_store_list['address_str'].tolist():
        new_ratio=fuzz.partial_ratio(store_SG,new_store_add.lower())
        if new_ratio>max_ratio:
            max_ratio=new_ratio
            best_match=new_store_add
            
    all_SG_SK_POI['matched_new_stores'][i]=best_match
    all_SG_SK_POI['highest_score'][i]=max_ratio

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [48]:
all_SG_SK_POI[all_SG_SK_POI['highest_score']>=60][['SG_address_str','matched_new_stores','highest_score']]

# No match with SafeGraph found, Go to Google Manually

,SG_address_str,matched_new_stores,highest_score
23,"5300 richmond avenue, houston, tx, 77056","22377 BELLAIRE BLVD #600, Richmond, TX, 77407",62
68,"7121 north west barry road, kansas city, mo, 6...","West Memorial Road, Oklahoma City, OK, 73114",65
140,"11414 montgomery road, cincinnati, oh, 45242","3251 VANDERCAR WAY, CINCINNATI, OH, 45209DMA",64
142,"9591 b colerain avenue, cincinnati, oh, 45251","3251 VANDERCAR WAY, CINCINNATI, OH, 45209DMA",66
155,"1620 west fm 646 road, league city, tx, 77573","West Memorial Road, Oklahoma City, OK, 73114",62
172,"324 west port plaza, saint louis, mo, 63146","6424 Chippewa St, St. Louis, MO, 63109",61
183,"440 c ohio pike, cincinnati, oh, 45255","3251 VANDERCAR WAY, CINCINNATI, OH, 45209DMA",63
255,"5921 south west third street, oklahoma city, o...","West Memorial Road, Oklahoma City, OK, 73114",69
363,"4475 forest park avenue, saint louis, mo, 63108","6424 Chippewa St, St. Louis, MO, 63109",61
377,"2700 childrens way, nashville, tn, 37212","6704 Whitestown Pkwy, Zionsville, IN, 46077",60


In [49]:
new_store_list

,storenumber,address_str
0,1646,"550 DURBIN PAVILION DRIVE,SUITE G107, Durbin P..."
1,1729,"22377 BELLAIRE BLVD #600, Richmond, TX, 77407"
2,1706,"West Memorial Road, Oklahoma City, OK, 73114"
3,1550,"6424 Chippewa St, St. Louis, MO, 63109"
4,1712,"1615 Ulster Ave, Lake Katrine, NY, 12449"
5,1742,"6704 Whitestown Pkwy, Zionsville, IN, 46077"
6,1719,"3251 VANDERCAR WAY, CINCINNATI, OH, 45209DMA"


In [53]:
google_dict={}
google_dict.update({"1646":{"Google_Add":"550 Durbin, Pavilion Dr Suite G107, FL 32259","Google_Coor":[30.0984943,-81.5218346]}})
google_dict.update({"1729":{"Google_Add":"Suite #600, 22377 Bellaire Blvd Suite #600, Richmond, TX 77407","Google_Coor":[29.6907498,-95.768674]}})
google_dict.update({"1706":{"Google_Add":"1001 W Memorial Rd, Oklahoma City, OK 73114","Google_Coor":[35.6094343,-97.5313474]}})
google_dict.update({"1550":{"Google_Add":"6424 Chippewa St, St. Louis, MO 63109","Google_Coor":[38.5935397,-90.2987058]}})
google_dict.update({"1712":{"Google_Add":"1615 Ulster Ave, Lake Katrine, NY 12449","Google_Coor":[41.9789269,-73.9897915]}})
google_dict.update({"1742":{"Google_Add":"6704 Whitestown Pkwy, Zionsville, IN 46077","Google_Coor":[39.9499312,-86.3418943]}})
google_dict.update({"1719":{"Google_Add":"3251 Vandercar Way, Cincinnati, OH 45209","Google_Coor":[39.1582807,-84.4271971]}}) # No SK store found


In [60]:
new_store_list=pd.read_csv("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20190311/new_store_appared.csv",dtype=str)
new_store_list['Latitude']=new_store_list['Latitude'].astype(float)
new_store_list['Longitude']=new_store_list['Longitude'].astype(float)
new_store_list['dist']=np.nan
for i in range(len(new_store_list)):
    client_coor=[new_store_list['Latitude'][i],new_store_list['Longitude'][i]]
    Google_coor=google_dict[new_store_list['storenumber'][i]]['Google_Coor']
    dist=haversine(client_coor,Google_coor)*1000
    new_store_list['dist'][i]=dist
    
    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [65]:
new_store_list['QC_level']=np.where(new_store_list['dist']<=30,"Google_in_30_meters",np.where(new_store_list['dist']<=100,"Google_30_to_100_meters","Google_above_100_meters"))

new_store_list_1=new_store_list[new_store_list['dist']<=30]
new_store_list_2=new_store_list[new_store_list['dist']>30]

new_store_list_1['assigned_lat']=new_store_list_1['Latitude']
new_store_list_1['assigned_long']=new_store_list_1['Longitude']
new_store_list_1['assigned_source']="Client_list"



new_store_list_2['assigned_lat']=new_store_list_2['storenumber'].apply(lambda x: google_dict[x]['Google_Coor'][0])
new_store_list_2['assigned_long']=new_store_list_2['storenumber'].apply(lambda x: google_dict[x]['Google_Coor'][1])
new_store_list_2['assigned_source']="Google_Map"

new_store_list=new_store_list_1.append(new_store_list_2)

new_store_list['updated_date']=str(datetime.datetime.now().date())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

In [64]:
google_dict['1729']['Google_Coor']

[29.6907498, -95.768674]

In [69]:
a=new_store_list.columns.tolist()
a.remove("dist")
new_store_list=new_store_list[a+["dist"]]

In [72]:
import os

os.getcwd()

'/home/jian/Projects/Smoothie_King/weekly_TA_updates'

In [73]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20190311/new_store_updates_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

new_store_list.to_excel(writer,"New_Stores",index=False)
writer.save()